In [2]:
! pip install -q kaggle
#from google.colab import files
#files.upload()  # upload kaggle.json with API token
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows


  0%|          | 0.00/13.4G [00:00<?, ?B/s]
  0%|          | 1.00M/13.4G [00:00<24:59, 9.56MB/s]
  0%|          | 3.00M/13.4G [00:00<22:34, 10.6MB/s]
  0%|          | 5.00M/13.4G [00:00<22:07, 10.8MB/s]
  0%|          | 7.00M/13.4G [00:00<21:45, 11.0MB/s]
  0%|          | 9.00M/13.4G [00:00<22:13, 10.7MB/s]
  0%|          | 11.0M/13.4G [00:01<21:53, 10.9MB/s]
  0%|          | 13.0M/13.4G [00:01<21:51, 10.9MB/s]
  0%|          | 15.0M/13.4G [00:01<21:38, 11.0MB/s]
  0%|          | 17.0M/13.4G [00:01<21:33, 11.1MB/s]
  0%|          | 19.0M/13.4G [00:01<21:26, 11.1MB/s]
  0%|          | 21.0M/13.4G [00:02<21:24, 11.1MB/s]
  0%|          | 23.0M/13.4G [00:02<22:13, 10.7MB/s]
  0%|          | 25.0M/13.4G [00:02<23:09, 10.3MB/s]
  0%|          | 26.0M/13.4G [00:02<23:30, 10.1MB/s]
  0%|          | 27.0M/13.4G [00:02<23:23, 10.2MB/s]
  0%|          | 28.0M/13.4G [00:02<25:48, 9.24MB/s]
  0%|          | 30.0M/13.4G [00:02<24:22, 9.78MB/s]
  0%|          | 32.0M/13.4G [00:03<23:17, 10.2MB/s]
 

In [ ]:
! mkdir twits
! unzip './ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip' -d twits

     -------------------------------------- 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 199.7/199.7 kB 11.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f2554b8895127a8558ef84cda3043f94df7fd65eaf863ca503feff9b6e257875
  Stored in directory: c:\users\anton\appdata\local\pip\cache\wheels\51\c8\18\298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [6]:
!rm /content/ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

"rm" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


# Project 1: Finding similar items

The task is to implement from scratch a detector of pairs of similar tweets, considering the text column of the dataset and selecting tweets written in a given language.


In [3]:
import findspark
findspark.init()
import pyspark

In [4]:
conf = pyspark.SparkConf().set('spark.ui.port', '4050')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession.builder.master('local[*]').getOrCreate()

In [5]:
spark

In [1]:
import numpy as np
import pandas as pd
import csv
import os
import re

======================================================

**Use only one of two following cells of code**

Read and load rdd and df of one particular file

In [6]:
filename = './twits/0820_UkraineCombinedTweetsDeduped.csv.gzip'
small_twits_df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
en_small_twits_df = small_twits_df[small_twits_df['language'] == 'en']
en_small_twits_df = en_small_twits_df.reset_index(drop=True)
data = en_small_twits_df[['tweetid','text']]
#spark = pyspark.sql.SparkSession.builder.appName('pd_to_rdd').getOrCreate()
rdd = spark.createDataFrame(data).rdd
#rdd.take(3)

Load df from directory

In [6]:
file_list = '' # list of loaded files
rdd = None
for file in os.scandir('./twits/'):
    if not os.path.isdir(file):
        print(file)
        file_list = file_list + file.path + '/n'
        twits_df = pd.read_csv(file, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
        twits_df = twits_df[twits_df['language'] == 'en']
        twits_df = twits_df.reset_index(drop=True)
        data = twits_df[['tweetid','text']]
        if rdd is None:
            #spark = pyspark.sql.SparkSession.builder.appName('pd_to_rdd').getOrCreate()
            rdd = spark.createDataFrame(data).rdd
        else:
            rdd1 = spark.createDataFrame(data).rdd
            rdd = rdd.union(rdd1)

<DirEntry '0819_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0820_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0821_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0822_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0823_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0824_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0825_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0826_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0827_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0828_UkraineCombinedTweetsDeduped.csv.gzip'>
<DirEntry '0829_UkraineCombinedTweetsDeduped.csv.gzip'>


UnionRDD[98] at union at <unknown>:0

## Shingling

2 possible methods:

1.   Token as char
2.   Token as word

I try Token as word here.

Tokenize:

1.   remove emoji
2.   split in words
3.   remove stopwords



In [7]:
!pip install emoji
from emoji import EMOJI_DATA, replace_emoji # EMOJI_DATA is a list of emojies

I use NLTK only to take list of stopwords

In [8]:
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_en = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Additionaly need to add URL removal from text (solution from https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python by Lee Martin)

In [9]:
def remove_urls(vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def tokenize(text):
  s = replace_emoji(text, replace='')
  s = remove_urls(s)
  if s != '':
    #words = re.split(r'[^\w'+removelist+']', s.lower())         # uncoment this if need to distinguish between hashtag and not hashtag words
    words = re.split(r'\W', s.lower())
    return [w for w in words if not w in stop_en and w != '']

#print('The World Health Organization (#WHO) says that two existing treatments dramatically reduced deaths from #Ebola and should be given to people of all ages suffering from the often-fatal hemorrhagic disease.\nhttps://t.co/iHF0FPULBU')
#print(remove_urls(en_small_twits_df.iloc[0]['text']))

Create n-grams of words for each line

In [10]:
def n_gram(tokens, n):
  if len(tokens) < n:
    l = [''] * (n-len(tokens))
    return tokens + l
  l = []
  for i in range(len(tokens)-n+1):
    l.append(' '.join(tokens[i:i+n]))
  return l

#n_gram(['a','b','c','d'],2)

**! List of shingles may contain similar shingles that can be in merged together, but i will not do it. Merge can be done only in case of k=2. Example of this shingles: 'russia ukraine' and 'ukraine russia'**

### Min-hashing and LSH

Create matrix of min-hash

Hash function: take list of shingles of one twitt, transform it to a int value

In [12]:
def string_hash(text, p, n):
  h = np.uint32(5381) #Bernstein's hash function djb2
  for i in range(len(text)):
    h = ((h*33) + ord(text[i])) % p
  return h % n

In [14]:
def generate_primes(n, m):
  #generates list of prime numbers larger than m
  #it (if i made all correct) have to return same list for same input
  primes = []
  num = m + 1 if m % 2 == 0 else m + 2 # Take first odd number grater than m
  while len(primes) < n:
    for i in range(3, int(num ** 0.5) + 1, 2): # no need to check if num is even (is always odd).
      # check if num is prime
      if num % i == 0:
        break
    else: #if num is prime (for is not breaked)
      primes.append(num)
    num += 2 # take next odd number to check
  return primes

#generate_primes(100, 2**32)

Signature of twitt: For each hash function calcolate hash value of each shingle, take minimum.

In [16]:
def signature(text, n, m, primes): 
  #text - list of shingles in one record, n - num of hash functins, m - max value of hash
  # for example i use n = 100, m = 2**32

  #primes = generate_primes(n, m)

  signature = [np.inf]*n
  for i, p in enumerate(primes):
    for shingle in text:
      h = string_hash(shingle, p, m)
      if h < signature[i]:
        signature[i] = h
  return signature

**Optional**

Jaccard similarity funtion:

In [17]:
#jaccard similarity
def jaccard(a,b):
  intersection = len(list(set(a).intersection(b)))
  union = (len(set(a)) + len(set(b))) - intersection
  return float(intersection) / union

**LSH**

we need to divide signature made of 100 minhash values in b bands of r rows (enough to specify b that 100%b = 0, for example b = 20) and put it in k baskets (b*k total)

For 200 minhash values keep b = 20, r = 10, s = 0.75

For 100 minhash values keep b = 20, r = 5, s = 0.54

$s = (1/b)^{1/r}$

In [18]:
def lsh(text, b, k):
  #return list of baskets indexes that this recort corrensponds to.
  l = [None]*b
  r = round(len(text)/b)
  for i in range(b):
    try:
      # take part of signeture of batch, sum it and mod k to find a bucet index from k possible backets
      # add i*k to separate backets of different batches.
      l[i] = (np.sum(text[r*i:r*(i+1)]) % k) + i*k
    except:
      print(i)
      raise Exception('LSH Error')
  return l

In [2]:
#List of parameters that were used to calculate LSH

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash

primes = generate_primes(N_HASH, MAX_HASH)

In [11]:
shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))

Prepare LSH from rdd of twitts

In [18]:
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list).cache()

**Optional**

collect all LSH buckets in cache

In [24]:
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print(collect_lsh_s[0])
print(end_time)

(408, [1560778639343587328, 1560795505000595456, 1560807732059545601, 1560810931789053952, 1560817894732300289, 1560882204926160897, 1560884969525886976, 1560891814617395200, 1560898735172878337, 1560914368728268801, 1560925610037444609, 1560927621441064960, 1560931108598030341, 1560932889793019904, 1560940622978207744, 1560954634134605825, 1560981826680012803, 1560988496327761922, 1560997234707091470, 1561001375785443328, 1561007785369899014, 1561027237956182018, 1561027596829212673, 1561041096037261313, 1561069833109872647, 1561096187457183745, 1561098894519459840, 1561112622359003137, 1561114220934176774, 1561114850838958081, 1561115191898648576])
224.0410134792328


In [65]:
import sys
sys.getsizeof(collect_lsh_s)

173016

## Find similar items

Count number of occurence of twitts in buckets that contain searched twitt

Filter results that have at least *s* occurences (from 1 to BANDS *check for costans in the beginning of notebook*)

If searched twitt is also in a LSH, result will also contain that twitt id with BANDS value as similatiry.

In [ ]:
filename = './twits/0820_UkraineCombinedTweetsDeduped.csv.gzip'
small_twits_df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
en_small_twits_df = small_twits_df[small_twits_df['language'] == 'en']
en_small_twits_df = en_small_twits_df.reset_index(drop=True)

In [76]:
# Random twitt from df
search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
print(search_record)
import time
start_time = time.time()
search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
print("signature count time: " + str(time.time() - start_time))
start_time = time.time()
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
print("search time: " + str(time.time() - start_time))
print(answer)

tweetid                                  1561111025822289920
text       @lilygrutcher An attempted Espionage or sabota...
Name: 18275, dtype: object
0.07699799537658691
14.049001216888428
[(1561111025822289920, 20)]


In [20]:
# search by twitt id
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1561010740189974529].iloc[0][['tweetid','text']]
print(search_record)
search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=10).collect()
print(answer)

tweetid                                  1561010740189974529
text       #Leaders #AI #transformation\n🎯#Fox #Business:...
Name: 11888, dtype: object
[(1560942577016258561, 20), (1561010740189974529, 20)]


## Optional: Calculate and save LSH of dataset 

Create LSH textfile

Made 1 lsh file for 9 twitter csv files.

(Wanted 10, but because i made an error in code 9. This is not critical)

In [ ]:
total_files = 0
for file in os.scandir('./twits/'):
  if not os.path.isdir(file):
    total_files+=1

i = 0
j = 0
lsh_file_list = ''

for file in os.scandir('./twits/'):
    if not os.path.isdir(file):
        twits_df = pd.read_csv(file, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
        twits_df = twits_df[twits_df['language'] == 'en']
        twits_df = twits_df.reset_index(drop=True)
        data = twits_df[['tweetid','text']]
        if i == 0:
            rdd = spark.createDataFrame(data).rdd
        else:
            rdd1 = spark.createDataFrame(data).rdd
            rdd = rdd.union(rdd1)
        lsh_file_list = lsh_file_list + file.path + '\n'
        i+=1
        if i == 9:
            shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),2)))
            lsh_s = shingles.map(lambda x: (x[0],signature(x[1],100,2**32))).map(lambda x: (x[0], lsh(x[1], 20, 1000))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list).cache()
            collect_lsh_s = lsh_s.collect()
            filename = 'lsh_s_' + str(j) + '.txt'
            with open(filename, 'w') as f:
                for line in collect_lsh_s:
                    f.write(f"{line}\n")
            filename = 'lsh_filelist_' + str(j) + '.txt'
            with open(filename, 'w') as f:
                f.write(lsh_file_list)
            j+=1
            i = 0
            lsh_file_list = ''
            print("Done: " + str(j*9) + " / " + str(total_files))
if i != 0:
    shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),2)))
    lsh_s = shingles.map(lambda x: (x[0],signature(x[1],100,2**32))).map(lambda x: (x[0], lsh(x[1], 20, 1000))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list).cache()
    collect_lsh_s = lsh_s.collect()
    filename = 'lsh_s_' + str(j) + '.txt'
    with open(filename, 'w') as f:
        for line in collect_lsh_s:
            f.write(f"{line}\n")
    filename = 'lsh_filelist_' + str(j) + '.txt'
    with open(filename, 'w') as f:
        f.write(lsh_file_list)
    j+=1
    print("Done: " + str(j*9) + " / " + str(total_files))

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

Create list of twitts ID and file that contain it 

(Can be used if all twitter files are not loaded into a single rdd)

In [83]:
total_files = 0
for file in os.scandir('./twits/'):
  if not os.path.isdir(file):
    total_files+=1

i = 0
j = 0

for file in os.scandir('./twits/'):
    if not os.path.isdir(file):
        twits_df = pd.read_csv(file, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
        twits_df = twits_df[twits_df['language'] == 'en']
        twits_df = twits_df.reset_index(drop=True)
        data = twits_df['tweetid'].tolist()
        filepath = str(file.path)
        if i == 0:
            filelist = []
            for d in data:
                filelist.append((d,filepath))
        else:
            for d in data:
                filelist.append((d,filepath))
        i+=1
        if i == 9:
            filename = 'twitt_list_' + str(j) + '.txt'
            with open(filename, 'w') as f:
                for line in filelist:
                    f.write(f"{line}\n")
            j+=1
            i = 0
            print("Done: " + str(j*9) + " / " + str(total_files))
if i != 0: 
    filename = 'twitt_list_' + str(j) + '.txt'
    with open(filename, 'w') as f:
        for line in filelist:
            f.write(f"{line}\n")
    print("Done: " + str(j*9 + i) + " / " + str(total_files))

Done: 9 / 129
Done: 18 / 129
Done: 27 / 129
Done: 36 / 129


C:\Users\Anton\AppData\Local\Temp\ipykernel_27468\3213938757.py:11: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  twits_df = pd.read_csv(file, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)


Done: 45 / 129
Done: 54 / 129
Done: 63 / 129
Done: 72 / 129
Done: 81 / 129
Done: 90 / 129
Done: 99 / 129
Done: 108 / 129
Done: 117 / 129
Done: 126 / 129
Done: 129 / 129


Loads saved lsh

In [18]:
lsh_s = None
for file in os.scandir('./'):
    if 'lsh_s_' in file.name:
        if lsh_s is None:
            lsh_s = sc.textFile(file.path).map(lambda x: x.split('[')).map(lambda x: (re.findall(r'\d+', x[0]),re.findall(r'\d+', x[1]))).map(lambda x: (int(x[0][0]),x[1]))
        else:
            lsh_s1 = sc.textFile(file.path).map(lambda x: x.split('[')).map(lambda x: (re.findall(r'\d+', x[0]),re.findall(r'\d+', x[1]))).map(lambda x: (int(x[0][0]),x[1]))
            lsh_s = lsh_s.union(lsh_s1).reduceByKey(lambda x, y: x + y)
lsh_s = lsh_s.sortBy(lambda x: x[0])
#collect_lsh_s = lsh_s.cache().collect()

Load list of twitts and files that contains it

In [7]:
records = None
for file in os.scandir('./'):
    if 'twitt_list_' in file.name:
        if records is None:
            records = sc.textFile(file.path).map(lambda x: x.split(',')).map(lambda x: (re.findall(r'\d+', x[0]),x[1].split("'")[1])).map(lambda x: (int(x[0][0]),x[1]))
        else:
            records1 = sc.textFile(file.path).map(lambda x: x.split(',')).map(lambda x: (re.findall(r'\d+', x[0]),x[1].split("'")[1])).map(lambda x: (int(x[0][0]),x[1]))
            records = records.union(records1)

### Optional: search simmilarity by known twitt id (requires list of twitts and files that contains it as a rdd)

Load text by twitt ID

In [6]:
def load_text_byID(twitt_id, records):
    filepath = records.lookup(twitt_id)[0]
    twits_df = pd.read_csv(filepath, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
    return twits_df[twits_df['tweetid'] == twitt_id].iloc[0]['text']

In [21]:
load_text_byID(1561012415461998603, records)

'Fantastic. 👏🏻👏🏻👏🏻👏🏻👏🏻👏🏻👏🏻👏🏻\n#SlavaUkraini #StandWithUkraine ✊🏻 \nhttps://t.co/fOx1PKbnnI'

Search similar twitts

In [22]:
def search_and_print(tweet_id, tweet_text, lsh_s, records, similarity):
    if similarity > b:
        print("Maximum similarity cannot exeed number of bashes: " + str(b))
        return None
    search_signature = (tweet_id,lsh(signature(n_gram(tokenize(tweet_text),TOKENS),N_HASH,MAX_HASH), BANDS, BUCKETS))
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=similarity).collect()
    print('original tweet:')
    print('id: ' + str(tweet_id))
    print('text: ' + tweet_text)
    print('=====================')
    for ans in answer:
        print('sim to ' + str(tweet_id) + ' and ' + str(ans[0]) + ': ' + str(ans[1]))
        text = load_text_byID(int(ans[0]), records)
        print('Jaccard: ' + str(jaccard(tokenize(tweet_text),tokenize(text))
        print(text)
        print('--------------------------------')

In [83]:
# Search and print text of similar tweets with a target tweetid 1561010740189974529 and at least 5 same buckets
search_and_print(1561010740189974529, en_small_twits_df[en_small_twits_df['tweetid'] == 1561010740189974529].iloc[0]['text'], lsh_s, records, 5)

sim to 1561010740189974529 and 1560942577016258561: 20
#Leaders #AI #transformation
🎯#Fox #Business: #China, #Russia 'new world order' is inescapable: Gen. #Keane
https://t.co/v0LSI1K4n7
https://t.co/aPJAt5VGgL
https://t.co/xuM07rKJAn
https://t.co/MrxjBS3uCF
https://t.co/58GMkZa1KB
https://t.co/NFgMyVni4t
https://t.co/gR3XRl3Ujp
--------------------------------
sim to 1561010740189974529 and 1561010740189974529: 20
#Leaders #AI #transformation
🎯#Fox #Business: #China, #Russia 'new world order' is inescapable: Gen. #Keane
https://t.co/t59o2WxXue
https://t.co/aRAOJLPqaT
https://t.co/kJgngbf7lK
https://t.co/vwIuYNWYkv
https://t.co/YhtzADeKF9
https://t.co/g4c0oPZbUW
https://t.co/ID11JAo8o0
--------------------------------


## Experiments

In [6]:
import time
filename = './twits/0820_UkraineCombinedTweetsDeduped.csv.gzip'
small_twits_df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
en_small_twits_df = small_twits_df[small_twits_df['language'] == 'en']
en_small_twits_df = en_small_twits_df.reset_index(drop=True)
data = en_small_twits_df[['tweetid','text']]
#spark = pyspark.sql.SparkSession.builder.appName('pd_to_rdd').getOrCreate()
rdd = spark.createDataFrame(data).rdd
#rdd.take(3)

start_time = time.time()
primes = generate_primes(N_HASH, MAX_HASH)
shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),2)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1],N_HASH,MAX_HASH,primes))).map(lambda x: (x[0], lsh(x[1], 20, 1000))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
collect_lsh_s = lsh_s.collect()
print((time.time() - start_time))
print(rdd.count())

In [17]:
import time
i = 0
for file in os.scandir('./twits/'):
    if not os.path.isdir(file):
        if '0820' in file.name or '0821' in file.name:
            twits_df = pd.read_csv(file, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
            twits_df = twits_df[twits_df['language'] == 'en']
            twits_df = twits_df.reset_index(drop=True)
            data = twits_df[['tweetid','text']]
            if i == 0:
                rdd = spark.createDataFrame(data).rdd
            else:
                rdd1 = spark.createDataFrame(data).rdd
                rdd = rdd.union(rdd1)
            i+=1
        if i == 2:
            start_time = time.time()
            primes = generate_primes(N_HASH, MAX_HASH)
            shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),2)))
            lsh_s = shingles.map(lambda x: (x[0],signature(x[1],N_HASH,MAX_HASH,primes))).map(lambda x: (x[0], lsh(x[1], 20, 1000))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
            collect_lsh_s = lsh_s.collect()
            print((time.time() - start_time))
            break
print(rdd.count())

443.23213624954224


In [81]:
import time
i = 0
for file in os.scandir('./twits/'):
    if not os.path.isdir(file):
        if '0820' in file.name or '0821' in file.name or '0822' in file.name:
            twits_df = pd.read_csv(file, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
            twits_df = twits_df[twits_df['language'] == 'en']
            twits_df = twits_df.reset_index(drop=True)
            data = twits_df[['tweetid','text']]
            if i == 0:
                rdd = spark.createDataFrame(data).rdd
            else:
                rdd1 = spark.createDataFrame(data).rdd
                rdd = rdd.union(rdd1)
            i+=1
        if i == 3:
            start_time = time.time()
            primes = generate_primes(N_HASH, MAX_HASH)
            shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),2)))
            lsh_s = shingles.map(lambda x: (x[0],signature(x[1],N_HASH,MAX_HASH,primes))).map(lambda x: (x[0], lsh(x[1], 20, 1000))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
            collect_lsh_s = lsh_s.collect()
            print((time.time() - start_time))
            break
print(rdd.count())

643.3270268440247


Count mean of search time

In [6]:
filename = './twits/0820_UkraineCombinedTweetsDeduped.csv.gzip'
small_twits_df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
en_small_twits_df = small_twits_df[small_twits_df['language'] == 'en']
en_small_twits_df = en_small_twits_df.reset_index(drop=True)
data = en_small_twits_df[['tweetid','text']]
#spark = pyspark.sql.SparkSession.builder.appName('pd_to_rdd').getOrCreate()
rdd = spark.createDataFrame(data).rdd
#rdd.take(3)

In [22]:
BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print("LSH calculation time:")
print(end_time)
import sys
print("Size of LSH cache:")
print(sys.getsizeof(collect_lsh_s))

print('===========================')
tt = 0.0
ts = 0.0
for i in range(100):
    search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
    start_time = time.time()
    search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
    end_time = time.time()
    #print((end_time - start_time))
    tt += (end_time - start_time)
    start_time = time.time()
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
    end_time = time.time()
    #print((end_time - start_time))
    ts += (end_time - start_time)
    
print('------------///////////----------')
print((tt/100.0),(ts/100.0))
#    print(answer)
#    print('\n\n')

235.82238268852234
173016
0.06602811813354492
12.328951597213745
0.042999982833862305
12.909142017364502
0.008015871047973633
12.613465785980225
0.06397581100463867
12.774462699890137
0.06199932098388672
12.24168610572815
0.026026487350463867
12.629358768463135
0.0429990291595459
13.038583040237427
0.02702045440673828
13.134390830993652
0.06199216842651367
12.731963396072388
0.017000436782836914
12.593986749649048
0.00899815559387207
12.671968698501587
0.04699850082397461
12.666496753692627
0.011004447937011719
12.770840644836426
0.06495332717895508
12.730679035186768
0.027022838592529297
12.301457166671753
0.04401755332946777
12.809852123260498
0.025997161865234375
12.624209880828857
0.03499865531921387
13.02623176574707
0.06199908256530762
12.876291513442993
0.050991058349609375
12.5556058883667
0.04999399185180664
13.047108888626099
0.03396940231323242
12.51486873626709
0.05199432373046875
13.005348682403564
0.03599810600280762
12.3309485912323
0.0570220947265625
12.519616842269897


In [23]:
BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print("LSH calculation time:")
print(end_time)
import sys
print("Size of LSH cache:")
print(sys.getsizeof(collect_lsh_s))

print('===========================')
tt = 0.0
ts = 0.0
for i in range(100):
    search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
    start_time = time.time()
    search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
    end_time = time.time()
    #print((end_time - start_time))
    tt += (end_time - start_time)
    start_time = time.time()
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
    end_time = time.time()
    #print((end_time - start_time))
    ts += (end_time - start_time)
    
print('------------///////////----------')
print((tt/100.0),(ts/100.0))
#    print(answer)
#    print('\n\n')

218.38458490371704
16184
0.005997896194458008
12.824039936065674
0.025026559829711914
12.654755115509033
0.020982980728149414
12.852445602416992
0.03099513053894043
13.366101026535034
0.033976078033447266
13.075906753540039
0.013999223709106445
12.814243793487549
0.06599974632263184
12.91880178451538
0.016972780227661133
12.812581777572632
0.03101944923400879
13.392468214035034
0.04499483108520508
13.060641765594482
0.03202486038208008
12.574079275131226
0.018025875091552734
12.951092720031738
0.06099653244018555
12.821863412857056
0.030998945236206055
13.026828527450562
0.08499574661254883
12.832362651824951
0.052971839904785156
12.53528618812561
0.01898193359375
12.512084722518921
0.0690000057220459
12.553094148635864
0.030976057052612305
13.08420181274414
0.056015968322753906
12.64391040802002
0.04197549819946289
13.098552703857422
0.03803062438964844
13.001233100891113
0.05602884292602539
12.677452325820923
0.0330195426940918
13.208306550979614
0.0599980354309082
12.654842376708984

In [24]:
BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print("LSH calculation time:")
print(end_time)
import sys
print("Size of LSH cache:")
print(sys.getsizeof(collect_lsh_s))

print('===========================')
tt = 0.0
ts = 0.0
for i in range(100):
    search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
    start_time = time.time()
    search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
    end_time = time.time()
    #print((end_time - start_time))
    tt += (end_time - start_time)
    start_time = time.time()
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
    end_time = time.time()
    #print((end_time - start_time))
    ts += (end_time - start_time)
    
print('------------///////////----------')
print((tt/100.0),(ts/100.0))
#    print(answer)
#    print('\n\n')

421.06221747398376
173016
0.11499977111816406
12.673866987228394
0.12402653694152832
12.437915802001953
0.09903120994567871
12.366593837738037
0.1289989948272705
12.523196458816528
0.08100771903991699
12.91687536239624
0.11900067329406738
12.628182888031006
0.02602672576904297
12.472917318344116
0.4389984607696533
12.349596500396729
0.014998912811279297
12.46495795249939
0.03800010681152344
12.784342050552368
0.07899165153503418
12.593489170074463
0.06899642944335938
12.63657832145691
0.1030282974243164
12.68916630744934
0.11502885818481445
12.343005657196045
0.10002756118774414
12.850990295410156
0.03499960899353027
12.759755373001099
0.08701586723327637
12.456097841262817
0.07700133323669434
12.868533611297607
0.05202913284301758
12.75425672531128
0.10801935195922852
12.548707246780396
0.14099931716918945
12.484606504440308
0.10400247573852539
12.596319913864136
0.09200072288513184
12.699769735336304
0.07703518867492676
13.01785135269165
0.06299829483032227
12.809865713119507
0.12199

In [25]:
BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print("LSH calculation time:")
print(end_time)
import sys
print("Size of LSH cache:")
print(sys.getsizeof(collect_lsh_s))

print('===========================')
tt = 0.0
ts = 0.0
for i in range(100):
    search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
    start_time = time.time()
    search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
    end_time = time.time()
    #print((end_time - start_time))
    tt += (end_time - start_time)
    start_time = time.time()
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
    end_time = time.time()
    #print((end_time - start_time))
    ts += (end_time - start_time)
    
print('------------///////////----------')
print((tt/100.0),(ts/100.0))
#    print(answer)
#    print('\n\n')

220.5789942741394
85176
0.07200121879577637
13.114999294281006
0.030002832412719727
12.992558002471924
0.05802154541015625
12.86987566947937
0.0200197696685791
12.529254674911499
0.024971485137939453
12.411368370056152
0.06303286552429199
12.712156534194946
0.09300112724304199
12.820615291595459
0.05500316619873047
12.890031337738037
0.022993087768554688
12.472544193267822
0.05899858474731445
12.582959175109863
0.014000892639160156
12.800443649291992
0.04201984405517578
13.11407732963562
0.06399941444396973
12.51577615737915
0.07901334762573242
12.505379915237427
0.041024208068847656
12.612588167190552
0.027996540069580078
12.617884397506714
0.004997968673706055
13.208641767501831
0.025002002716064453
12.620797395706177
0.0490264892578125
12.35099720954895
0.010020732879638672
12.655038833618164
0.010029315948486328
12.692516803741455
0.020996809005737305
13.178159952163696
0.04397082328796387
12.700817346572876
0.042029380798339844
12.649909257888794
0.04099631309509277
12.67753291130

In [26]:
BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print("LSH calculation time:")
print(end_time)
import sys
print("Size of LSH cache:")
print(sys.getsizeof(collect_lsh_s))

print('===========================')
tt = 0.0
ts = 0.0
for i in range(100):
    search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
    start_time = time.time()
    search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
    end_time = time.time()
    #print((end_time - start_time))
    tt += (end_time - start_time)
    start_time = time.time()
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
    end_time = time.time()
    #print((end_time - start_time))
    ts += (end_time - start_time)
    
print('------------///////////----------')
print((tt/100.0),(ts/100.0))
#    print(answer)
#    print('\n\n')

220.43481874465942
219064
0.011996746063232422
12.762577533721924
0.05599856376647949
12.55783486366272
0.026021480560302734
12.510501146316528
0.06102156639099121
12.635893106460571
0.03299736976623535
12.550935983657837
0.020974159240722656
12.498881816864014
0.02300405502319336
12.275146484375
0.024998903274536133
13.70438528060913
0.020998477935791016
12.96507453918457
0.05899477005004883
12.351153135299683
0.046977996826171875
12.386858224868774
0.04101896286010742
12.865129709243774
0.01902031898498535
12.367221117019653
0.03403043746948242
12.986273765563965
0.03299832344055176
12.743830442428589
0.05200052261352539
12.447450876235962
0.055030107498168945
12.580687046051025
0.028025150299072266
12.360053539276123
0.008005380630493164
12.880360126495361
0.01802849769592285
12.284582138061523
0.03699779510498047
12.598032474517822
0.048998355865478516
12.519467830657959
0.08199739456176758
12.494811534881592
0.042998313903808594
13.169189929962158
0.06299996376037598
12.8746330738

In [27]:
BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print("LSH calculation time:")
print(end_time)
import sys
print("Size of LSH cache:")
print(sys.getsizeof(collect_lsh_s))

print('===========================')
tt = 0.0
ts = 0.0
for i in range(100):
    search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
    start_time = time.time()
    search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
    end_time = time.time()
    #print((end_time - start_time))
    tt += (end_time - start_time)
    start_time = time.time()
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
    end_time = time.time()
    #print((end_time - start_time))
    ts += (end_time - start_time)
    
print('------------///////////----------')
print((tt/100.0),(ts/100.0))
#    print(answer)
#    print('\n\n')

139.33454084396362
173016
0.01201009750366211
12.977538347244263
0.0069959163665771484
12.708106517791748
0.00899815559387207
12.309601545333862
0.03297066688537598
12.408851385116577
0.007997989654541016
12.564412832260132
0.01998615264892578
12.675443172454834
0.00699925422668457
12.952386856079102
0.034975528717041016
12.744630098342896
0.007028341293334961
12.942699670791626
0.006997823715209961
12.848982095718384
0.015998125076293945
12.688977003097534
0.029015064239501953
13.162891149520874
0.03499937057495117
12.805161714553833
0.030992507934570312
12.719957828521729
0.03399658203125
12.771106719970703
0.01800060272216797
12.85887336730957
0.020972728729248047
13.143899202346802
0.01696920394897461
12.678240537643433
0.03999757766723633
12.962343454360962
0.015028238296508789
13.003205060958862
0.023999452590942383
13.13677453994751
0.024999380111694336
13.322359561920166
0.018000364303588867
13.002450942993164
0.009996891021728516
12.723389625549316
0.019024372100830078
12.5916

In [28]:
BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)
import time
start_time = time.time()
collect_lsh_s = lsh_s.collect()
end_time = time.time() - start_time
print("LSH calculation time:")
print(end_time)
import sys
print("Size of LSH cache:")
print(sys.getsizeof(collect_lsh_s))

print('===========================')
tt = 0.0
ts = 0.0
for i in range(100):
    search_record = en_small_twits_df[['tweetid','text']].sample(1).iloc[0]
    start_time = time.time()
    search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
    end_time = time.time()
    #print((end_time - start_time))
    tt += (end_time - start_time)
    start_time = time.time()
    answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).collect()
    end_time = time.time()
    #print((end_time - start_time))
    ts += (end_time - start_time)
    
print('------------///////////----------')
print((tt/100.0),(ts/100.0))
#    print(answer)
#    print('\n\n')

292.1011734008789
173016
0.07999634742736816
12.367831707000732
0.05499839782714844
13.085931301116943
0.08302807807922363
12.545076370239258
0.07399415969848633
12.64473032951355
0.026998519897460938
12.709683895111084
0.025026321411132812
12.571649551391602
0.04899859428405762
13.512852430343628
0.04797053337097168
12.911753177642822
0.00699925422668457
12.818846702575684
0.05999875068664551
12.87521767616272
0.02397298812866211
13.029248714447021
0.040000200271606445
13.135928630828857
0.06699037551879883
12.937897205352783
0.02902531623840332
12.992859840393066
0.05302691459655762
13.000807523727417
0.062033653259277344
12.991647243499756
0.05699753761291504
13.011254787445068
0.06302928924560547
12.651792287826538
0.04402518272399902
12.719950199127197
0.0670015811920166
12.976734399795532
0.024998903274536133
12.935436010360718
0.010996103286743164
13.087275266647339
0.06799769401550293
12.483688831329346
0.08299756050109863
12.765823125839233
0.0410306453704834
12.61150693893432

Manual check of similar tweets:

In [6]:
filename = './twits/0820_UkraineCombinedTweetsDeduped.csv.gzip'
small_twits_df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
en_small_twits_df = small_twits_df[small_twits_df['language'] == 'en']
en_small_twits_df = en_small_twits_df.reset_index(drop=True)
data = en_small_twits_df[['tweetid','text']]
#spark = pyspark.sql.SparkSession.builder.appName('pd_to_rdd').getOrCreate()
rdd = spark.createDataFrame(data).rdd
#rdd.take(3)

List of tweetid that i used as target over search from 0820 file:

In [ ]:
[1561115380600283136, 1561126718601105410, 1560970279572176896, 1560916346828996608, 1560872710565433344, 1561027561022423043, 1561076241851097091, 1560790174740746243]

1 - original (no FP)

2 - more false possitive (much more with sim 3, some additional sim 4 FP)

3 - only 100%

4 - only 100%

5 - found new similar tweet '1561115373801390081'. No FP. Changed values of old one. Miss '1560785746130137088' record

6 - Very strong FP results (have max sim)

7 - only 100%

In [20]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1561115380600283136].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1561115380600283136, 20, 1.0), (1560830509982224384, 4, 0.875), (1560785746130137088, 3, 0.5263157894736842), (1560853141708365824, 4, 0.7222222222222222)]
20 100 2 100
-------------------------
[(1560807328617615360, 3, 0.0), (1561023184874651648, 3, 0.0), (1561065324048523264, 3, 0.0), (1561115380600283136, 20, 1.0), (1560818517141864448, 3, 0.029411764705882353), (1560830509982224384, 4, 0.875), (1560853141708365824, 4, 0.7222222222222222), (1560890654053617664, 3, 0.0), (1561028268442124288, 3, 0.0), (1560896386232528896, 3, 0.02631578947368421), (1560781490635644928, 3, 0.0), (1560960862231228416, 3, 0.0), (1560889581155008512, 3, 0.0), (1561038229939396608, 3, 0.0), (1560785746130137088, 3, 0.5263157894736842), (1560893665316261888, 3, 0.0), (1561099416131768320, 3, 0.0), (1560852982970793985, 3, 0.0), (1560946650771853313, 3, 0.0), (1561005702004592641, 3, 0.0), (1560912967688126465, 3, 0.0), (1560780650986328065, 3, 0.0), (1560899449844

1 - original (no FP), one have sim = 11

2 - more false possitive (much more with sim 3)

3 - Same as (1), but lower sim of second tweet (in (1) have 11/20, here 4/20)

4 - Same as (1), but lower sim of second tweet (in (1) have 11/20, here 3/10 (~6/20))

5 - Same as (1) (in (1) second tweet have 11/20, here 14/25 (~11.2/20))

6 - Very strong FP results (have max sim), miss sim in (1) 11/20

7 - only 100%

In [21]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1561126718601105410].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1561116707489456130, 11, 0.7142857142857143), (1561126718601105410, 20, 1.0)]
20 100 2 100
-------------------------
[(1560805264298745856, 3, 0.0), (1560930740405157888, 3, 0.02857142857142857), (1561107894086406144, 3, 0.043478260869565216), (1560974837257883648, 3, 0.0625), (1561114753690476544, 3, 0.11764705882352941), (1560950910427201537, 3, 0.1111111111111111), (1560986835806986241, 3, 0.037037037037037035), (1560994714723225601, 3, 0.0), (1561035097662541825, 3, 0.04), (1561062679019769857, 3, 0.05263157894736842), (1561116707489456130, 11, 0.7142857142857143), (1561126718601105410, 20, 1.0), (1560893470889361410, 3, 0.0), (1561036089464332290, 3, 0.0), (1561090184057421834, 3, 0.043478260869565216), (1561054798224449538, 3, 0.03225806451612903), (1560964612476096515, 3, 0.0), (1561058512427524100, 3, 0.05263157894736842), (1560995446662656005, 3, 0.041666666666666664)]
20 1000 2 200
-------------------------
[(1561126718601105410, 20, 

1 - original (no FP), Lots of same tweets, one have text repetition in it (sim = 9)

2 - more false possitive (all with sim 3)

3 - Same as (1), but lower sim of not max sim tweet (in (1) have 9/20, here 4/20)

4 - Same as (1), but miss not max sim tweet (in (1) have 9/20)

5 - Same as (1) (in (1) not max sim tweet have 9/20, here 13/25 (~10.4/20))

6 - Very strong FP results (have max sim). (in (1) not max sim tweet have 9/20, here 20/20)

7 - Same as (1), but lower sim of not max sim tweet (in (1) have 9/20, here 3/20)

In [22]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1560970279572176896].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1560970279572176896, 20, 1.0), (1561001346211389440, 20, 1.0), (1561001375584161800, 20, 1.0), (1561001709769572352, 20, 1.0), (1561001899930882048, 20, 1.0), (1561001313340891137, 20, 1.0), (1561001737292754945, 9, 1.0), (1561001796516155393, 20, 1.0), (1561001974484848641, 20, 1.0), (1560970918188519426, 20, 1.0), (1560971086354911235, 20, 1.0), (1561001655684198403, 20, 1.0), (1561001449408143364, 20, 1.0)]
20 100 2 100
-------------------------
[(1560970279572176896, 20, 1.0), (1561001346211389440, 20, 1.0), (1561001375584161800, 20, 1.0), (1561001709769572352, 20, 1.0), (1561001899930882048, 20, 1.0), (1561051203500494848, 3, 0.0), (1561045941808795648, 3, 0.0), (1561046919824101376, 3, 0.0), (1561001313340891137, 20, 1.0), (1561001796516155393, 20, 1.0), (1561001974484848641, 20, 1.0), (1561001737292754945, 9, 1.0), (1560797633593184257, 3, 0.0), (1561005702004592641, 3, 0.0), (1561078561158545409, 3, 0.0), (1560970918188519426, 20, 1.0),

1 - original (no FP). only target tweet

2 - lot false possitive (sim = 3/4)

3 - only target tweet

4 - only target tweet

5 - only target tweet

6 - Have lots of similar tweets (they have same key words as target tweet: Pantsir-S1 (can be writed as Pancyr-S1), Sewastopol), Some ammount of FP

7 - only target tweet

In [23]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1560916346828996608].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1560916346828996608, 20, 1.0)]
20 100 2 100
-------------------------
[(1560886162759172096, 3, 0.0), (1560916346828996608, 20, 1.0), (1560937162677268480, 3, 0.03125), (1560827506978164736, 3, 0.10526315789473684), (1560945937782296576, 3, 0.043478260869565216), (1560976127992135680, 3, 0.02564102564102564), (1560989011027664896, 3, 0.041666666666666664), (1560851342498750464, 3, 0.0), (1560908287318659072, 3, 0.0), (1561070793689145344, 3, 0.038461538461538464), (1560919679476047872, 3, 0.04), (1560909244425281537, 3, 0.0), (1560918325894737921, 3, 0.06451612903225806), (1561029584115666945, 4, 0.041666666666666664), (1560926733171408897, 3, 0.07692307692307693), (1561018683828445185, 3, 0.0), (1561050438052392961, 3, 0.030303030303030304), (1561001522317729793, 3, 0.0), (1560826958396768257, 3, 0.0), (1560914536793874433, 3, 0.034482758620689655), (1560860588875534337, 3, 0.05), (1560970296131309570, 3, 0.0), (1560987779307376642, 3, 0.04761

1 - All tweets have same text, but different '@user'. All of them have sim 3-6 (exept target that have 20), have some ammount of false negative (see (2),(5), have 9 tweets)

2 - lot false possitive (sim = 3), found some other similar tweets

3 - only target tweet

4 - only target tweet

5 - Same as (1), but have more tweets (14 tweets)

6 - Have lots of similar tweets (same text, have sim > 10 ), Large ammount of FP, some ammount of FP with sim > 10. total results 294, 17 have same text with sim > 10

7 - Same as (1), but have less tweets (6 tweets)

In [24]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1560872710565433344].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1560872710565433344, 20, 1.0), (1560871349652193280, 6, 0.7142857142857143), (1560871537062080512, 6, 0.7142857142857143), (1560871622214828032, 4, 0.7142857142857143), (1560872209807470592, 5, 0.7142857142857143), (1560872515630960640, 4, 0.7142857142857143), (1560872980800208896, 3, 0.7142857142857143), (1560871041219837952, 3, 0.7142857142857143), (1560872600980824066, 4, 0.7142857142857143)]
20 100 2 100
-------------------------
[(1560872710565433344, 20, 1.0), (1560916833129021440, 3, 0.0), (1561124380536885248, 3, 0.0), (1560888073055084544, 3, 0.0), (1560950217440202752, 3, 0.0), (1560983930781609984, 3, 0.05555555555555555), (1560871622214828032, 4, 0.7142857142857143), (1560872209807470592, 5, 0.7142857142857143), (1560963458065174528, 3, 0.0), (1560864888049860608, 3, 0.0), (1560871349652193280, 6, 0.7142857142857143), (1560871537062080512, 6, 0.7142857142857143), (1560872515630960640, 4, 0.7142857142857143), (1560872980800208896, 3,

1 - only target tweet

2 - false possitive (sim = 3)

3 - only target tweet

4 - only target tweet

5 - only target tweet

6 - Have lots of FP with max or close simtotal results 520

7 - Have target tweet + 1 FP tweet with sim = 3 (ony folowing strings have something simmilar "from #Russia's hard" and "from #Russia had")

In [25]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1561027561022423043].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1561027561022423043, 20, 1.0)]
20 100 2 100
-------------------------
[(1560779589487583232, 3, 0.0), (1560858696292487168, 3, 0.0), (1560983713713754112, 3, 0.03571428571428571), (1561111095397556224, 3, 0.0), (1561121370775379968, 3, 0.0), (1560983606100725760, 3, 0.0), (1561031847643054080, 3, 0.0), (1560842666920669185, 3, 0.0), (1561057848884432898, 3, 0.0), (1561027561022423043, 20, 1.0)]
20 1000 2 200
-------------------------
[(1561027561022423043, 20, 1.0)]
10 1000 2 100
-------------------------
[(1561027561022423043, 10, 1.0)]
25 1000 2 100
-------------------------
[(1561027561022423043, 25, 1.0)]
20 1000 1 100
-------------------------
[(1560778754300968960, 20, 0.04081632653061224), (1560782087908884480, 20, 0.023255813953488372), (1560790436801028096, 20, 0.058823529411764705), (1560792608493953024, 20, 0.047619047619047616), (1560793086229446656, 20, 0.022222222222222223), (1560802368119136256, 20, 0.02702702702702703), (1560803

1 - only target tweet + 1 similar text tweet (sim = 16/20, differences: emojie, new lines, missing full hashtags or missing hashtag sighn ('#'), some punctuation and/or space (space between words) errors)

2 - same as (1) + false possitive results (sim = 3/4)

3 - same as (1), but second tweet have sim = 10/20 (in (1) 16/20)

4 - same as (1), but second tweet have sim = 6/10 (~12/20) (in (1) 16/20)

5 - same as (1), but second tweet have sim = 21/25 (~16.8/20) (in (1) 16/20)

6 - Have lots of FP with close to max sim (both tweets with sim = 20 are same as in (1)). total results 49

7 - same as (1), but second tweet have sim = 15/20 (in (1) 16/20)

In [19]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1561076241851097091].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1561076241851097091, 20, 1.0), (1561077604270186501, 16, 0.9523809523809523)]
20 100 2 100
-------------------------
[(1560801662163652608, 3, 0.02631578947368421), (1561058667121836032, 3, 0.037037037037037035), (1561123987845181440, 3, 0.02702702702702703), (1560829403340218368, 3, 0.0), (1560882556211445760, 3, 0.06666666666666667), (1561091589765619712, 3, 0.0), (1561108428419989504, 3, 0.06666666666666667), (1560974933819154432, 3, 0.034482758620689655), (1561052599469629440, 4, 0.03125), (1560786428371648512, 3, 0.03125), (1561042462193303552, 3, 0.0), (1560792608493953024, 3, 0.02564102564102564), (1560848110896701440, 3, 0.05128205128205128), (1560853232515026944, 3, 0.05128205128205128), (1560873414750978048, 3, 0.05128205128205128), (1560921085205483520, 3, 0.04878048780487805), (1560912032785989633, 3, 0.04), (1560874526233296897, 3, 0.0), (1561114850838958081, 3, 0.0), (1560910001522221058, 3, 0.0), (1560935410167152642, 3, 0.0), (1

1 - only target tweet + 1 similar text tweet (sim = 8/20, differences: @user)

2 - same as (1) + false possitive results (sim = 3/4)

3 - only target tweet

4 - only target tweet

5 - same as (1), but second tweet have sim = 10/25 (~8/20) (in (1) 8/20)

6 - same as (1), but second tweet have sim = 6/20 (in (1) 8/20)

7 - same as (1), but second tweet have sim = 4/20 (in (1) 8/20)

In [20]:
search_record = en_small_twits_df[en_small_twits_df['tweetid'] == 1560790174740746243].iloc[0][['text','tweetid']]

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')


BANDS = 20 # num of bands
BUCKETS = 100 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 100 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 200 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 2 200')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 10 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('10 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 25 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 2 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('25 1000 2 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 1 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 1 100')
print('-------------------------')
print(answer)
print('=========================')

BANDS = 20 # num of bands
BUCKETS = 1000 # num of buckets in each band (total = k*b)
TOKENS = 3 # num of shingles in k-gramm
N_HASH = 100 # num of hash functions
MAX_HASH = 2**32 # max value of hash
primes = generate_primes(N_HASH, MAX_HASH)

shingles = rdd.map(lambda s: (s[0], n_gram(tokenize(s[1]),TOKENS)))
lsh_s = shingles.map(lambda x: (x[0],signature(x[1], N_HASH, MAX_HASH, primes))).map(lambda x: (x[0], lsh(x[1], BANDS, BUCKETS))).map(lambda x: [(s, x[0]) for s in x[1]]).flatMap(lambda x: x).groupByKey().mapValues(list)

search_signature = (search_record['tweetid'],lsh(signature(n_gram(tokenize(search_record['text']),TOKENS),N_HASH,MAX_HASH, primes), BANDS, BUCKETS))
answer = lsh_s.filter(lambda x: x[0] in search_signature[1]).flatMap(lambda x: x[1]).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1]>=3).map(
    lambda x: (x[0],x[1],jaccard(tokenize(search_record['text']),tokenize(en_small_twits_df[en_small_twits_df['tweetid'] == x[0]].iloc[0]['text'])))).collect()
print('20 1000 3 100')
print('-------------------------')
print(answer)
print('=========================')

20 1000 2 100
-------------------------
[(1560784633637265409, 8, 0.7666666666666667), (1560790174740746243, 20, 1.0)]
20 100 2 100
-------------------------
[(1561089290863448064, 3, 0.0), (1560950749202358272, 4, 0.0), (1560925074294800384, 3, 0.04878048780487805), (1560784951984812032, 3, 0.0), (1560958839314386944, 3, 0.0), (1561032105760624640, 3, 0.0), (1560922239851184128, 3, 0.0), (1561067847392104449, 3, 0.0), (1560784633637265409, 8, 0.7666666666666667), (1560911435240296449, 3, 0.0), (1560857547594014721, 3, 0.0), (1561031233328627713, 3, 0.022727272727272728), (1561066820777390081, 3, 0.0), (1561008746201063425, 4, 0.023809523809523808), (1560794310328881154, 4, 0.0), (1560874145956503554, 3, 0.0), (1560853804303765506, 3, 0.0), (1560790174740746243, 20, 1.0), (1561043384986476547, 3, 0.0), (1561061319117361155, 3, 0.0), (1561065534531182597, 3, 0.0)]
20 1000 2 200
-------------------------
[(1560790174740746243, 20, 1.0)]
10 1000 2 100
-------------------------
[(156079017

To read text:

In [26]:
print(en_small_twits_df[en_small_twits_df['tweetid'] == 1561076241851097091].iloc[0]['text'])

Russia put"on pause"the holding of"referendums"
Most likely,the invaders did not expect the decisiveness of the Armed Forces of #Ukraine️. In Russia,they were shocked by the explosions in the #Crimea, they didn’t think that such a thing would ever happen.
#russiaisateroriststate https://t.co/kpt1JvSK7K


In [27]:
print(en_small_twits_df[en_small_twits_df['tweetid'] == 1561077604270186501].iloc[0]['text'])

🔎 #Russia put "on pause" the holding of "#referendums"

Most likely, the invaders did not expect the decisiveness of the Armed Forces of #Ukraine️. In Russia, they were shocked by the explosions in the #Crimea, they didn’t think that such a thing would ever happen https://t.co/4Hut7x2zIB


To read text from list

In [13]:
li = [(1561089290863448064, 3), (1560950749202358272, 4), (1560925074294800384, 3), (1560784951984812032, 3), (1560958839314386944, 3), (1561032105760624640, 3), (1560922239851184128, 3), (1561067847392104449, 3), (1560784633637265409, 8), (1560911435240296449, 3), (1560857547594014721, 3), (1561031233328627713, 3), (1561066820777390081, 3), (1561008746201063425, 4), (1560794310328881154, 4), (1560874145956503554, 3), (1560853804303765506, 3), (1560790174740746243, 20), (1561043384986476547, 3), (1561061319117361155, 3), (1561065534531182597, 3)]
for l in li:
    #if l[1] > 10:
        #if l[0] == 1561001737292754945:
        #    pass
    print(en_small_twits_df[en_small_twits_df['tweetid'] == l[0]].iloc[0]['text'])
    print(l[1])
    print('==========================')
print(len(li))

Very observant of him. It's been obvious to me for a long time too. #Putin #vladimir #Russians #USA #bureaucracy #uniparty #Elites #EliteCoup https://t.co/fqxeLpX9N4
3
Ring walk 22.30, it's been broadcast in #Ukraine for free, 

I think that #AnthonyJoshua is in for a tough time! 

Links to streaming in the Youtube clip attached.. 
#UsykJoshua2

https://t.co/n0BQz3iOhD
4
@martinplaut For half a century this is what you can give them, deteriorating life.
#TPLFisTheCause 4suffer, #war hunger, displacement, death..on #Tigray ppl for half a century.
#NoMore
3
Decorative pear Felt handmade https://t.co/KL0QycVakL via @EtsySocial  #giftideas #Ukraine #cushionforneedles #seamstressgift https://t.co/rEBJKIoxhf
3
I stand with #Ukraine! https://t.co/2ExYNzS67s
3
Quotes that no one should say/believe:

"I trust #Trump completely! #GOP forever!"

"I trust #Biden completely! #Dems forever!"

#BeCritcal #BeFair #BeResponsible #BeIndependent #TrustYourself #UnityIsPatriotic #TruthIsElusive
3
Rare Cup